In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import librosa 
import seaborn as sns
import tensorflow as tf


In [2]:
# Chargement des fichiers audios du dossier fan/test dans un dataframe
import os, sys
class File_charge:
    """
    La classe File_charge permet d'instancier un objet de type dataframe contenant les chemins d'accès des fichiers audio
    contenus dans chaque sous dossier du dataset
    
    Paramètres:
                path : Chemin d'accès au répetoire des fichiers audio
                expemple : path = "E:/dataset/fan/train/"    
    """
    def __init__(self, path):
        self.path = path 
        
    def load_file(self):
        """
        La fonction load_file retourne un dataframe constituer des chemins d'accès aux fichiers audio contenu 
        dans un sous dossier du dataset
        
        """
        
        dirs = os.listdir(self.path)

        df = list()
        for dir in dirs:
            df.append((self.path+"/"+dir))

        df = pd.DataFrame(df, columns = ['audio_file'])
        df = df.reset_index()
        return  df




In [3]:
# Fonction de chargement des fichiers audio
def load_audio(audio_path):
    return librosa.load(audio_path, sr=None)

In [4]:
def split_path(df_path, drop = True):
    """
    Transforme le df avec une colonne d'index + le path C:/blabla/machine/train/etc
    vers une séparation des colonnes en fonction des machines, ID, train/test etc...
    
    """
    machine, types, normality, ID, numero = [], [], [], [], []

    for path in df_path['audio_file'].tolist():
        machine.append(path.split('/')[-3])
        types.append(path.split('/')[-2])
        normality.append(path.split('/')[-1].split('_')[0])
        ID.append(path.split('/')[-1].split('_')[2])
        numero.append(path.split('/')[-1].split('_')[3][:-4])
        
    df_path['machine']= machine
    df_path['types'] = types
    df_path['normality'] = normality
    df_path['ID'] = ID
    df_path['numero'] = numero
    if drop:
        df_path.drop(columns=['audio_file', 'index'], inplace=True)

In [5]:
_, fe = librosa.load('E:/son_atypyque/dataset/valve/train/normal_id_00_00000000.wav', sr=None)

def logMelSpectrogram(audio, params, fe):
    stfts = librosa.stft(audio,n_fft = int(params['n_fft']),hop_length = int(params["frame_step"]),center = False).T
    power_spectrograms = np.real(stfts * np.conj(stfts))
    linear_to_mel_weight_matrix = librosa.filters.mel(sr=fe,n_fft=int(params['n_fft']) + 1,n_mels=params['num_mel_bins'],
                                fmin=params['lower_edge_hertz'],fmax=params['upper_edge_hertz']).T
    mel_spectrograms = np.tensordot(power_spectrograms,linear_to_mel_weight_matrix, 1)
    return (np.log(mel_spectrograms + 1e-8).astype(np.float16))

params = {'n_fft': 1024,'frame_step': 10240/64,'lower_edge_hertz': 0,'upper_edge_hertz': 8000,'num_mel_bins': 128}

In [ ]:
### Les tests du modèle ResNet ci-dessous se font sur la machine VALVE
### Seule la cellule ce-desous est nécessaire sur les 5 prochaines
### Pour faire des essais sur d'autres machines, il faudra lancer les cellules correspondantes

In [6]:
### VALVE ###

# Chargement du dataframe des chemins d'accès des audios contenus dans le dataset/train
df_valve_train = File_charge("E:/son_atypyque/dataset/valve/train")
df_valve_train = df_valve_train.load_file()

# Création d'un tableau des listes des amplitudes des échantillons audio d'entrainement de la machine "fan"
list_valve_train = list()
for i in range(len(df_valve_train)):
    list_valve_train.append(load_audio(df_valve_train.iloc[i,1])[0]/255)
    
# Chargement du dataframe des chemins d'accès des audios contenus dans le dataset/train
df_valve_test = File_charge("E:/son_atypyque/dataset/valve/test")
df_valve_test = df_valve_test.load_file()

# Création d'un tableau des listes des amplitudes des échantillons audio d'entrainement de la machine "fan"
list_valve_test = list()
for i in range(len(df_valve_test)):
    list_valve_test.append(load_audio(df_valve_test.iloc[i,1])[0]/255)
    

split_path(df_valve_train)
split_path(df_valve_test)

LMS_Valve_train = []

for table in list_valve_train:
    LMS_Valve_train.append(logMelSpectrogram(table, params, fe))
    
LMS_Valve_test = []

for table in list_valve_test:
    LMS_Valve_test.append(logMelSpectrogram(table, params, fe))

In [9]:
### TOYCONVEYOR ###

# Chargement du dataframe des chemins d'accès des audios contenus dans le dataset/train
df_ToyConveyor_train = File_charge("E:/son_atypyque/dataset/ToyConveyor/train/")
df_ToyConveyor_train = df_ToyConveyor_train.load_file()

# Création d'un tableau des listes des amplitudes des échantillons audio d'entrainement de la machine "fan"
list_ToyConveyor_train = list()
for i in range(len(df_ToyConveyor_train)):
    list_ToyConveyor_train.append(load_audio(df_ToyConveyor_train.iloc[i,1])[0]/255)

# Chargement du dataframe des chemins d'accès des audios contenus dans le dataset/train
df_ToyConveyor_test = File_charge("E:/son_atypyque/dataset/ToyConveyor/test/")
df_ToyConveyor_test = df_ToyConveyor_test.load_file()

# Création d'un tableau des listes des amplitudes des échantillons audio d'entrainement de la machine "fan"
list_ToyConveyor_test = list()
for i in range(len(df_ToyConveyor_test)):
    list_ToyConveyor_test.append(load_audio(df_ToyConveyor_test.iloc[i,1])[0]/255)


split_path(df_ToyConveyor_train)
split_path(df_ToyConveyor_test)


LMS_TConv_train = []

for table in list_ToyConveyor_train:
    LMS_TConv_train.append(logMelSpectrogram(table, params, fe))

LMS_TConv_test = []

for table in list_ToyConveyor_test:
    LMS_TConv_test.append(logMelSpectrogram(table, params, fe))  

In [127]:
### FAN ###

# Chargement du dataframe des chemins d'accès des audios contenus dans le dataset/train
df_fan_train = File_charge("E:/son_atypyque/dataset/fan/train")
df_fan_train = df_fan_train.load_file()

# Création d'un tableau des listes des amplitudes des échantillons audio d'entrainement de la machine "fan"
list_fan_train = list()
for i in range(len(df_fan_train)):
    list_fan_train.append(load_audio(df_fan_train.iloc[i,1])[0]/255)

# Chargement du dataframe des chemins d'accès des audios contenus dans le dataset/train
df_fan_test = File_charge("E:/son_atypyque/dataset/fan/test")
df_fan_test = df_fan_test.load_file()

# Création d'un tableau des listes des amplitudes des échantillons audio d'entrainement de la machine "fan"
list_fan_test = list()
for i in range(len(df_fan_test)):
    list_fan_test.append(load_audio(df_fan_test.iloc[i,1])[0]/255)
    
    
split_path(df_fan_train)    
split_path(df_fan_test)

LMS_Fan_train = []

for table in list_fan_train:
    LMS_Fan_train.append(logMelSpectrogram(table, params, fe))

LMS_Fan_test = []

for table in list_fan_test:
    LMS_Fan_test.append(logMelSpectrogram(table, params, fe))

In [128]:
### PUMP ###

# Chargement du dataframe des chemins d'accès des audios contenus dans le dataset/train
df_pump_train = File_charge("E:/son_atypyque/dataset/pump/train")
df_pump_train = df_pump_train.load_file()

# Création d'un tableau des listes des amplitudes des échantillons audio d'entrainement de la machine "pump"
list_pump_train = list()
for i in range(len(df_pump_train)):
    list_pump_train.append(load_audio(df_pump_train.iloc[i,1])[0]/255)

# Chargement du dataframe des chemins d'accès des audios contenus dans le dataset/train
df_pump_test = File_charge("E:/son_atypyque/dataset/pump/test")
df_pump_test = df_pump_test.load_file()

# Création d'un tableau des listes des amplitudes des échantillons audio d'entrainement de la machine "pump"
list_pump_test = list()
for i in range(len(df_pump_test)):
    list_pump_test.append(load_audio(df_pump_test.iloc[i,1])[0]/255)
    
    
split_path(df_pump_train)    
split_path(df_pump_test)

LMS_Pump_train = []

for table in list_pump_train:
    LMS_Pump_train.append(logMelSpectrogram(table, params, fe))
    
LMS_Pump_test = []

for table in list_pump_test:
    LMS_Pump_test.append(logMelSpectrogram(table, params, fe))

In [129]:
### SLIDER ###

# Chargement du dataframe des chemins d'accès des audios contenus dans le dataset/train
df_slider_train = File_charge("E:/son_atypyque/dataset/slider/train")
df_slider_train = df_slider_train.load_file()

# Création d'un tableau des listes des amplitudes des échantillons audio d'entrainement de la machine "slider"
list_slider_train = list()
for i in range(len(df_slider_train)):
    list_slider_train.append(load_audio(df_slider_train.iloc[i,1])[0]/255)

# Chargement du dataframe des chemins d'accès des audios contenus dans le dataset/train
df_slider_test = File_charge("E:/son_atypyque/dataset/slider/test")
df_slider_test = df_slider_test.load_file()

# Création d'un tableau des listes des amplitudes des échantillons audio d'entrainement de la machine "slider"
list_slider_test = list()
for i in range(len(df_slider_test)):
    list_slider_test.append(load_audio(df_slider_test.iloc[i,1])[0]/255)
    
    
split_path(df_slider_train)    
split_path(df_slider_test)

LMS_Slider_train = []

for table in list_slider_train:
    LMS_Slider_train.append(logMelSpectrogram(table, params, fe))

LMS_Slider_test = []

for table in list_slider_test:
    LMS_Slider_test.append(logMelSpectrogram(table, params, fe))

In [7]:
# Données d'entrainement sur les valves: 
# ID = 00 => Classe positive
# ID = 02 => Classe négative
# ID = autre => Non pris en compte

# Pour faire des essais sur d'autres machines :
# Remplacer LMS_Valve_train par :
# Fan : LMS_Fan_train
# ToyConveyor : LMS_TConv_train
# Pump : LMS_Pump_train
# Slider : LMS_Slider_train

X_train = list()
y_train = list()

for i in range(len(LMS_Valve_train)):
    if df_valve_train.ID[i] =='00':
        X_train.append(LMS_Valve_train[i])
        y_train.append(1)
    elif df_valve_train.ID[i] == '02':
        X_train.append(LMS_Valve_train[i])
        y_train.append(0)
        
X_train = np.array(X_train).reshape((-1, 994, 128, 1))

ValueError: cannot reshape array of size 119344384 into shape (994,128,1)

In [8]:
# Données de test :
# ID = 00
# ======= Machine normale => Classe positive
# ======= Machine anormale => Classe négative
# ID = autre => Non pris en compte

# Pour faire des essais sur d'autres machines :
# Remplacer LMS_Valve_test par :
# Fan : LMS_Fan_test
# ToyConveyor : LMS_TConv_test
# Pump : LMS_Pump_test
# Slider : LMS_Slider_test

X_test = list()
y_test = list()

for i in range(len(LMS_Valve_test)):
    if df_valve_test.ID[i] == '00':
        X_test.append(LMS_Valve_test[i])
        if (df_valve_test.normality[i] =='normal'):
            y_test.append(1)
        else:
            y_test.append(0)

X_test = np.array(X_test).reshape((-1, 994, 128, 1))

In [9]:
count1 = 0
count0 = 0

for i in y_train:
    if i==1:
        count1 +=1
    else:
        count0 +=1
        
print('y_train :')
print('classe positive :', count1)
print('classe négative :', count0)
print('ratio :', (count1/(count1+count0)*100))

y_train :
classe positive : 891
classe négative : 608
ratio : 59.43962641761175


In [11]:
count1 = 0
count0 = 0

for i in y_test:
    if i==1:
        count1 +=1
    else:
        count0 +=1
        
print('y_test :')
print('classe positive :', count1)
print('classe négative :', count0)
print('ratio :', (count1/(count1+count0)*100))

y_test :
classe positive : 100
classe négative : 119
ratio : 45.662100456621005


In [12]:
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

def transform(x, y):
    # Redimensionnement des données
    img = tf.reshape(x, (994, 128, 1))
    # Augmentation des données
    img = tf.image.random_crop(img, [25, 25, 1])
    # On retourne les images redimensionnées 
    return tf.image.resize(img, (994, 128)), y

dataset = dataset.map(transform)
dataset = dataset.shuffle(200).batch(16)

In [ ]:
####################
### MODEL RESNET ###
####################

In [13]:
from keras.layers import Input, Add, Dense, Activation
from keras.layers import ZeroPadding2D
from keras.layers import BatchNormalization
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import AveragePooling2D
from keras.layers import MaxPooling2D
from keras.models import Model

from keras.initializers import glorot_uniform
from keras.utils.vis_utils import model_to_dot

In [14]:
### Identity block ###

def identity_block(X, f, filters, stage, block):
    # Définition du nom de la base
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) +block + '_branch'
    
    # Récupération des filtres
    F1, F2, F3 = filters
    
    # Initialisation du shortcut
    X_shortcut = X
    
    # Chemin principal - 1
    X = Conv2D(filters = F1, 
               kernel_size = (1, 1), 
               strides = (1, 1), 
               padding = 'valid', 
               name = conv_name_base + '2a',
               kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3,
                           name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Chemin principal - 2    
    X = Conv2D(filters = F2,
               kernel_size = (f, f), 
               strides = (1, 1),
               padding = 'same',
               name = conv_name_base + '2b',
               kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3,
                           name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Chemin principal - 3   
    X = Conv2D(filters = F3,
               kernel_size = (1, 1),
               strides = (1,1),
               padding = 'valid',
               name = conv_name_base + '2c',
               kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3,
                           name = bn_name_base + '2c')(X)
    
    # Ajout du shortcut
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X


In [15]:
### Convolutional Block ###

def convolutional_block(X, f, filters, stage, block, s = 2):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Récupération des filtres
    F1, F2, F3 = filters
    
    # Initialisation du shortcut
    X_shortcut = X


    # Chemin principal - 1
    X = Conv2D(F1,
               (1, 1),
               strides = (s,s),
               name = conv_name_base + '2a',
               kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3,
                           name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Chemin principal - 2   
    X = Conv2D(filters=F2,
               kernel_size=(f, f),
               strides=(1, 1),
               padding='same',
               name=conv_name_base + '2b',
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3,
                           name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Chemin principal - 3 
    X = Conv2D(filters=F3,
               kernel_size=(1, 1),
               strides=(1, 1),
               padding='valid',
               name=conv_name_base + '2c',
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3,
                           name=bn_name_base + '2c')(X)

    
    # Définition du shortcut
    X_shortcut = Conv2D(F3,
                        (1, 1),
                        strides = (s,s),
                        name = conv_name_base + '1',
                        kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3,
                                    name = bn_name_base + '1')(X_shortcut)

    # Ajout du shortcut
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X


In [16]:
### Model ResNet ###

def ResNet50(input_shape = (994, 128, 1), classes = 2):   
    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input)
    
    # Etape 1
    X = Conv2D(64, (7, 7),
               strides = (2, 2),
               name = 'conv1',
               kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3,
                           name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Etape 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Etape 3
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Etape 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Etape 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # Pooling.
    X = AveragePooling2D((2, 2), name='avg_pool')(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    

    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model


In [17]:
ROWS = 994
COLS = 128
CHANNELS = 1
CLASSES = 2

In [25]:
model_RESNET = ResNet50(input_shape = (ROWS, COLS, CHANNELS),
                 classes = CLASSES)

In [26]:
model_RESNET.compile(optimizer='adam',
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

In [20]:
y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

In [21]:
history_RESNET = model_RESNET.fit(X_train, y_train, epochs = 1, batch_size = 32)

47/47 [==============================] - 1209s 24s/step - loss: 7.4644 - accuracy: 0.5170


In [27]:
model_RESNET.fit(dataset, 
                 epochs = 3,
                 batch_size =  32)

Epoch 1/3
94/94 [==============================] - 1145s 12s/step - loss: 7.2552 - accuracy: 0.8859
Epoch 2/3
94/94 [==============================] - 1167s 12s/step - loss: 4.8066 - accuracy: 0.8779
Epoch 3/3
94/94 [==============================] - 1176s 13s/step - loss: 4.1508 - accuracy: 0.8506


In [29]:
test_pred = model_RESNET.predict(X_test)
preds = model_RESNET.evaluate(X_test, y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

7/7 [==============================] - 34s 5s/step - loss: 45677.7461 - accuracy: 0.5434
Loss = 45677.74609375
Test Accuracy = 0.543379008769989


In [55]:
print('Taux de valeurs négatives :', round(1- (sum(y_test)/len(y_test))[0], 4))
print('Précision :', round(preds[1], 4))

Taux de valeurs négatives : 0.5434
Précision : 0.5434


In [60]:
from sklearn import metrics

y_test_class = y_test.argmax(axis = 1)
test_pred_class = test_pred.argmax(axis = 1)

print(metrics.confusion_matrix(y_test, test_pred_class))

[[119   0]
 [100   0]]


In [ ]:
### Ici, l'ensemble des spectrogramme ont été considérés comme anormaux

In [24]:
import keras
keras.backend.clear_session()